In [ ]:
!pip install openpyxl

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score

from IPython.display import clear_output
%matplotlib inline


import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam 

import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

sns.set(font_scale=1.2)
sns.set_style(style='whitegrid')
device_num = 0

In [ ]:
df_train=pd.read_excel('es_AMI_TrainingSet_NEW.xlsx')

In [ ]:
df_train.head()

,id,tweet,misogynous,misogyny_category,target
0,1,Módulo 01 Invitados Directivos del Sindicato '...,0,0,0
1,2,"@Causafiebres Claro, es que ha puesto imbécil ...",0,0,0
2,3,@Nanchinho A ver si haces algo que sirva para ...,1,discredit,active
3,4,@Rafhyta86 @ElDesmadre__ Habe pasame el video ...,1,discredit,passive
4,5,@Otb_35 Que te calles so guarra.,1,dominance,active


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rtatman/pretrained-word-vectors-for-spanish

100% 0.98G/0.98G [00:10<00:00, 140MB/s]
100% 0.98G/0.98G [00:10<00:00, 101MB/s]


In [ ]:
! unzip pretrained-word-vectors-for-spanish.zip

Archive:  pretrained-word-vectors-for-spanish.zip
  inflating: SBW-vectors-300-min5.txt  


In [ ]:
train_data, test_data = train_test_split(df_train, test_size=0.2, random_state=1)
print("Train Data size:", len(train_data))
print("Test Data size", len(test_data))

Train Data size: 2645
Test Data size 662


In [ ]:
MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 300

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.tweet)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

Vocabulary Size : 9916


In [ ]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.tweet),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.tweet),
                       maxlen = MAX_SEQUENCE_LENGTH)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

Training X Shape: (2645, 300)
Testing X Shape: (662, 300)


In [ ]:
labels = train_data.misogynous.unique().tolist()

In [ ]:
y_train = train_data.misogynous
y_test = test_data.misogynous

In [ ]:
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

In [ ]:
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (2645, 1)
y_test shape: (662, 1)


In [ ]:
GLOVE_EMB = 'SBW-vectors-300-min5.txt'
EMBEDDING_DIM = 300
LR = 1e-3
BATCH_SIZE = 1024
EPOCHS = 1

In [ ]:
embeddings_index = {}

f = open(GLOVE_EMB)
for line in f:
  values = line.split()
  word = value = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

Found 1000654 word vectors.


In [ ]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                          EMBEDDING_DIM,
                                          weights=[embedding_matrix],
                                          input_length=MAX_SEQUENCE_LENGTH,
                                          trainable=False)

In [ ]:
batch_size = 8
epochs = 1
embed_size = 300
max_features = 9916
maxlen = 300


In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [ ]:
model = Sequential()
model.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

optimzer=Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 300)          2974800   
                                                                 
 lstm_8 (LSTM)               (None, 300, 128)          219648    
                                                                 
 lstm_9 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,245,969
Trainable params: 271,169
Non-trainable params: 2,974,800
_________________________________________________________________


In [ ]:
history = model.fit(x_train, y_train, batch_size = 8 , validation_data = (x_test,y_test), verbose=2)

331/331 - 350s - loss: 0.6625 - accuracy: 0.6038 - val_loss: 0.5979 - val_accuracy: 0.6949 - 350s/epoch - 1s/step


In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(x_train,y_train)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

83/83 [==============================] - 17s 198ms/step - loss: 0.6185 - accuracy: 0.6813
Accuracy of the model on Training Data is -  68.12854409217834 %
21/21 [==============================] - 4s 180ms/step - loss: 0.5979 - accuracy: 0.6949
Accuracy of the model on Testing Data is -  69.486403465271 %
